In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
path = "DataSets/belly_button_biodiversity.sqlite"
engine = create_engine("sqlite:///"+path, echo=False)
Base = automap_base()
Base.prepare(engine, reflect=True)
meta = Base.metadata
meta.reflect(bind=engine)
# Binding the SQLITE engine to the current Session (locking file)
session = Session(bind=engine).connection_callable

In [3]:
# Verify tables and structure from SQLite database
inspector = inspect(engine)
tbl_names = inspector.get_table_names()
i = 0 
for table in tbl_names:
    print(table)
    i = i+1
print("___________________________\nTotal of %s tables" %i)

otu
samples
samples_metadata
sqlite_sequence
___________________________
Total of 4 tables


In [4]:
tables = Base.classes.keys()
for table in tables:
    code_str = str("Base.classes."+str(table))
    globals() ["%s_table" %table] = eval(code_str)

In [5]:
OTU = Base.classes.otu
SAMPLES = Base.classes.samples
META = Base.classes.samples_metadata

In [6]:
OTU_COLS = []
columns = inspector.get_columns("otu")
for col in columns:
    OTU_COLS.append(col["name"])

In [7]:
SAMPLE_COLS = []
columns = inspector.get_columns("samples")
for col in columns:
    SAMPLE_COLS.append(col["name"])

In [8]:
META_COLS = []
columns = inspector.get_columns("samples_metadata")
for col in columns:
    META_COLS.append(col["name"])

In [9]:
OTU_df = pd.read_sql("SELECT * FROM otu", con=engine, columns=[OTU_COLS], index_col="otu_id")
SAMPLES_df = pd.read_sql("SELECT * FROM samples", con=engine, columns=[SAMPLE_COLS], index_col="otu_id")
META_ALL_df = pd.read_sql("SELECT * FROM samples_metadata", con=engine, columns=[META_COLS])
META_df = pd.read_sql("SELECT AGE, UPPER(BBTYPE) as BBTYPE, UPPER(ETHNICITY) as ETHNICITY, UPPER(GENDER) as GENDER, UPPER(LOCATION) as LOCATION, SAMPLEID FROM samples_metadata", con=engine, columns=[META_COLS])
META_WFREQ = pd.read_sql("SELECT WFREQ, SAMPLEID FROM samples_metadata", con=engine, columns=[META_COLS])

In [10]:
sample_names = SAMPLES_df.columns
sample_list_names = []
for col in sample_names:
    sample_list_names.append(col)

# @app.route("/names")
names = sample_list_names[0:]

In [11]:
otu_desc = []
for item in OTU_df["lowest_taxonomic_unit_found"]:
    otu_desc.append(item)

# @app.route("/otu")
otu = otu_desc

In [12]:
meta_ = META_df
meta_["index"]= "BB_"+meta_["SAMPLEID"].astype(str)
meta_.set_index("index", inplace=True)

# @app.route("/metadata/<sample>")
metadata = meta_.to_dict("index")

In [19]:
metadata

{'BB_940': {'AGE': 24,
  'BBTYPE': 'I',
  'ETHNICITY': 'CAUCASIAN',
  'GENDER': 'F',
  'LOCATION': 'BEAUFORT/NC',
  'SAMPLEID': 940},
 'BB_941': {'AGE': 34,
  'BBTYPE': 'I',
  'ETHNICITY': 'CAUCASIAN/MIDLEASTERN',
  'GENDER': 'F',
  'LOCATION': 'CHICAGO/IL',
  'SAMPLEID': 941},
 'BB_943': {'AGE': 49,
  'BBTYPE': 'I',
  'ETHNICITY': 'CAUCASIAN',
  'GENDER': 'F',
  'LOCATION': 'OMAHA/NE',
  'SAMPLEID': 943},
 'BB_944': {'AGE': 44,
  'BBTYPE': 'I',
  'ETHNICITY': 'EUROPEAN',
  'GENDER': 'M',
  'LOCATION': 'NEWHAVEN/CT',
  'SAMPLEID': 944},
 'BB_945': {'AGE': 48,
  'BBTYPE': 'I',
  'ETHNICITY': 'CAUCASIAN',
  'GENDER': 'F',
  'LOCATION': 'PHILIDELPHIA/PA',
  'SAMPLEID': 945},
 'BB_946': {'AGE': 42,
  'BBTYPE': 'I',
  'ETHNICITY': 'CAUCASIAN',
  'GENDER': 'F',
  'LOCATION': 'DEERFIELD/MA',
  'SAMPLEID': 946},
 'BB_947': {'AGE': 49,
  'BBTYPE': 'I',
  'ETHNICITY': 'CAUCASIAN',
  'GENDER': 'M',
  'LOCATION': 'CHAPELHILL/NC',
  'SAMPLEID': 947},
 'BB_948': {'AGE': 20,
  'BBTYPE': 'I',
  'ETHNI

In [13]:
wfreq_ = META_WFREQ
wfreq_["index"] = "BB_"+wfreq_["SAMPLEID"].astype(str)
wfreq_.drop(["SAMPLEID"], axis=1, inplace=True)
wfreq_.set_index("index", inplace=True)

# @app.route("/wfreq/<sample>")
wfreq = wfreq_.to_dict("index")   #  wfreq["BB_940"]["WFREQ"]

In [14]:
otu_id_sort = OTU_df.sort_index(ascending=False)

In [17]:
import random as rnd
user_val = rnd.choice(names) 
sample_values = SAMPLES_df
temp_samp_df = sample_values.sort_values(user_val, ascending=False)
temp_list = temp_samp_df[user_val][:10]
samp_data = temp_list.to_dict()
new_dict = {
    "otu_ids":list(samp_data.keys()),
    "sample_values":list(samp_data.values())
}

In [18]:
new_dict

{'otu_ids': [1795, 2539, 2722, 944, 1167, 2964, 3450, 2651, 2419, 909],
 'sample_values': [382, 145, 101, 79, 69, 46, 42, 40, 30, 29]}